In [5]:
import torch
import torch.nn as nn
import pandas
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(10)
dtype = torch.FloatTensor

In [8]:
def generate_real():
    real_data = torch.FloatTensor(
            [np.random.uniform(0.8, 1.0),
            np.random.uniform(0.0, 0.2),
            np.random.uniform(0.8, 1.0),
            np.random.uniform(0.0, 0.2)])     # CPU上32位浮点型
    return real_data

In [11]:
class Discriminator(nn.Module):
    def __init__(self):
        # 初始化PyTorch父类
        super().__init__()

        # 定义神经网络层
        self.model = nn.Sequential(
                        nn.Linear(4, 3),
                        nn.Sigmoid(),
                        nn.Linear(3, 1),
                        nn.Sigmoid()
        )

        # 创建损失函数
        self.loss_function = nn.MSELoss()

        # 创建优化器, 使用随机梯度下降
        self.optimiser = torch.optim.SGD(self.parameters(), 
                                            lr=0.01)
        
    def forward(self,inputs):
        return self.model(inputs)
    


In [13]:
aa = Discriminator()
aa1 = generate_real()
print(aa1)
aa(aa1)


tensor([0.8338, 0.0177, 0.9371, 0.1907])


tensor([0.5817], grad_fn=<SigmoidBackward0>)